In [ ]:
# System Operations
import os
import sys

# Mathematical Operations
import numpy as np
from scipy.io import loadmat # Works for MATLAB versions up to and including 7.1
from numpy.fft import fft
from numpy.fft import fftfreq
from numpy.fft import ifft
from scipy.signal import TransferFunction
from scipy.signal import bode
import matplotlib.pyplot as plt

In [ ]:
class MyTransfer():
    def __init__(self, num, denom):
        self.num = num
        self.denom = denom
        self.num_order = len(num)
        self.denom_order = len(denom)
        if(0 == self.num_order):
            sys.error("Enter a proper numerator.")
        if(0 == self.denom_order):
            sys.error("Enter a proper denominator.")
        
    def __call__(self, freq):
        num_temp = 0.0
        denom_temp = 0.0
        for i in range(self.num_order):
            num_temp += self.num[i]*np.power(freq, self.num_order - i - 1)
        for i in range(self.denom_order):
            denom_temp += self.denom[i]*np.power(freq, self.denom_order - i - 1)
        return num_temp/denom_temp

In [ ]:
path = os.path.join(os.getcwd(), "training2017/A00001.mat")
data = loadmat(path)
ekg = np.squeeze(np.array(data["val"]), axis = 0)
ekg = ekg/np.max(ekg)

In [ ]:
fs = 300 # Samples per second
dt = 1/fs # Sampling spacing
num_seconds = 30 # Ten second sample period
t = np.linspace(0, num_seconds, num_seconds*fs)

In [ ]:
# Take the Fourier Transform
components = fft(ekg)
real_components = components.real
freq = fftfreq(ekg.shape[-1], d = dt)

In [ ]:
figs, axs = plt.subplots(2)

axs[0].plot(t, ekg)
axs[0].set_title("Sinusoid")
axs[0].set_xlabel("Time [s]")
axs[0].set_ylabel("EKG")

axs[1].plot(freq, real_components)
# axs[1].set_xlim(left = 0)
axs[1].set_title("Fourier Domain")
axs[1].set_xlabel("Frequency [Hz]")
axs[1].set_ylabel("Magnitude")

plt.tight_layout()

In [ ]:
for i in range(freq.shape[-1]):
    if(0 > freq[i]):
        cutoff = i
        break

In [ ]:
positive_freq = freq[:cutoff]
positive_components = components[:cutoff]

In [ ]:
#### Low Pass Filter Design ####

In [ ]:
omega_naught = 5
num = [1]
denom = [1/omega_naught, 1]
transfer = TransferFunction(num, denom)

In [ ]:
#### Bode Plot ####

In [ ]:
w, mag, phase = bode(transfer)
fig, ax = plt.subplots(2)

# Bode magnitude plot
ax[0].semilogx(w, mag)
ax[0].set_title("Magnitude")
ax[0].set_xlabel("Log(Angular Frequency)")
ax[0].set_ylabel("Magnitude")

# Bode phase plot
ax[1].semilogx(w, phase)
ax[1].set_title("Phase")
ax[1].set_xlabel("Log(Angular Frequency)")
ax[1].set_ylabel("Phase [Rad]")
plt.tight_layout()

In [ ]:
#### Apply Filter ####

In [ ]:
myFunc = MyTransfer(num, denom)
transfer_coeffs = []
for i in range(positive_freq.shape[-1]):
    transfer_coeffs.append(myFunc(positive_freq[i]))
transfer_coeffs = np.array(transfer_coeffs)
filtered_components = transfer_coeffs*positive_components
filtered_ekg = ifft(filtered_components)

In [ ]:
figs, axs = plt.subplots(2)

axs[0].plot(t[:cutoff], filtered_ekg)
axs[0].set_title("Sinusoid")
axs[0].set_xlabel("Time [s]")
axs[0].set_ylabel("EKG")

axs[1].plot(positive_freq, filtered_components)
# axs[1].set_xlim(left = 0)
axs[1].set_title("Fourier Domain")
axs[1].set_xlabel("Frequency [Hz]")
axs[1].set_ylabel("Magnitude")

plt.tight_layout()

In [ ]:
#### Reference Documentation ####
# https://numpy.org/doc/stable/reference/generated/numpy.fft.fft.html#numpy.fft.fft
# https://numpy.org/doc/stable/reference/generated/numpy.fft.fftfreq.html
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.TransferFunction.html
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.bode.html
# https://www.youtube.com/user/ControlLectures/playlists